In [4]:
import sys
sys.path.append('../../')

In [6]:
%matplotlib inline

import os

import logging

from skimage.io import imread, imsave
import numpy as np
import pandas as pd

import shutil
import warnings
import tempfile
import subprocess
import linecache

from multiprocessing import Pool

import matplotlib.pyplot as plt

import cv2

from tqdm.auto import tqdm

import tifffile

from lib.utils import init_script, array2patches

In [24]:
TIFFTILES2JPG_BIN = '../../../tiff/tifftiles2jpg/tifftiles2jpg'

In [8]:
wsi_path = "/mnt/HDDData/pdata/train_images/"
wsi_masks_path = "/mnt/HDDData/pdata/train_label_masks/"
wsi_csv_path = "/mnt/HDDData/pdata/train.csv"
patches_path = "/mnt/HDDData/pdata/processed/patches_512/"
patches_csv_path = "/mnt/HDDData/pdata/processed/patches_512.csv"

patch_sz = 512

threads = 4

In [9]:
for d in ["imgs", "masks"]:
    os.makedirs(os.path.join(patches_path, d), exist_ok=True)

In [10]:
wsi_df = pd.read_csv(wsi_csv_path)

In [11]:
wsi_df

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0
...,...,...,...,...
10611,ffd2841373b39792ab0c84cccd066e31,radboud,0,negative
10612,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5
10613,ffe06afd66a93258f8fabdef6044e181,radboud,0,negative
10614,ffe236a25d4cbed59438220799920749,radboud,2,3+4


In [12]:
wsi_df['isup_grade'].unique()

array([0, 4, 1, 3, 5, 2])

In [13]:
have_masks = True

In [25]:
def process_wsi(df_row):
    def get_exception():
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        linecache.checkcache(filename)
        line = linecache.getline(filename, lineno, f.f_globals)
        return 'EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj)   
    
    rows = []
    
    try:
        img_id = df_row.image_id
        print(img_id)
        
        with tempfile.TemporaryDirectory() as tmp_path:
            result = subprocess.run([TIFFTILES2JPG_BIN, 
                                     os.path.join(wsi_path, f"{img_id}.tiff"), 
                                     '0', tmp_path], 
                                     stdout=subprocess.PIPE)

            if result.returncode != 0:
                raise Exception(f"TIFFTILES2JPG error. Return code: {result.returncode}. \n" + 
                                result.stdout.decode("utf-8"))

            if have_masks:
                img = tifffile.imread(os.path.join(wsi_masks_path, f"{img_id}_mask.tiff"))
                mask_patches = array2patches(img, patch_sz)

            for y in range(mask_patches.shape[0]):
                for x in range(mask_patches.shape[1]):
                    t_patch_path = os.path.join(tmp_path, f"{y}_{x}.jpeg")
                    patch = imread(t_patch_path)
                    mask_patch = mask_patches[y, x, ..., 0]

                    if patch.mean() != 255:
                        row = [img_id, df_row.data_provider, df_row.isup_grade, 
                               df_row.gleason_score, y, x]

                        mask_sz = np.prod(mask_patch.shape)

                        row += [(mask_patch == i).sum() / mask_sz for i in range(6)]

                        with warnings.catch_warnings():
                            warnings.filterwarnings("ignore", message=".*low contrast image.*")
                            shutil.copyfile(t_patch_path, os.path.join(patches_path, 
                                                                       f"imgs/{img_id}_{y}_{x}.jpeg"))
                            imsave(os.path.join(patches_path, 
                                                f"masks/{img_id}_{y}_{x}.png"), mask_patch)
                        rows.append(row)
                
        err = None
                
    except:
        err = get_exception()
    
    return rows, err

In [33]:
patches_rows = []

In [36]:
# Те что не были обработаны скриптом из-за нехватки памяти
# df_row = wsi_df[wsi_df['image_id'] == 'a1b7055a495ac4a160336b66acc62b4b'].iloc[0]
#df_row = wsi_df[wsi_df['image_id'] == 'f948e5f2b0a49af2c0a7f3f74093262e'].iloc[0]

In [37]:
df_row

image_id         f948e5f2b0a49af2c0a7f3f74093262e
data_provider                          karolinska
isup_grade                                      1
gleason_score                                 3+3
Name: 10315, dtype: object

In [38]:
res = process_wsi(df_row)

f948e5f2b0a49af2c0a7f3f74093262e


In [39]:
rows, err = res

In [40]:
len(rows)

499

In [41]:
err

In [42]:
patches_rows.extend(rows)

In [43]:
len(patches_rows)

684

In [47]:
patches_csv_path = "/mnt/SSDData/pdata/processed/patches_512.csv"

In [48]:
!cp {patches_csv_path} '/mnt/SSDData/pdata/processed/patches_512_copy.csv'

In [49]:
patches_df0 = pd.read_csv(patches_csv_path)

In [50]:
patches_df1 = pd.DataFrame(patches_rows,
                          columns=['image_id', 'data_provider',
                                   'isup_grade', 'gleason_score',
                                   'y', 'x'] + [f"label{i}"
                                                for i in range(6)])

# patches_df.to_csv(patches_csv_path, index=False)

In [51]:
patches_df = pd.concat([patches_df0, patches_df1])

In [53]:
patches_df0.shape

(2576045, 12)

In [52]:
patches_df.shape

(2576729, 12)

In [ ]:
patches_df.to_csv(patches_csv_path, index=False)

In [24]:
patches_rows = []
error_wsis = []

In [25]:
tmp_df = wsi_df.iloc[:10]

In [26]:
l_rows = []

with Pool(processes=threads) as pool:
    for rows, err in tqdm(pool.imap_unordered(process_wsi, 
                                              (r[1] for r in tmp_df.iterrows())), 
                          total=len(tmp_df)):
        if err:
            error_wsis.append({img_id: err})
        else:
            patches_rows.extend(rows)

000920ad0b612851f8e01bcc880d9b3d
0005f7aaab2800f6170c399693a96917
001c62abd11fa4b57bf7a6c603a11bb9


0018ae58b01bdadc8e347995b69f99aa
001d865e65ef5d2579c190a0e0350d8f
002a4db09dad406c85505a00fb6f6144
003046e27c8ead3e3db155780dc5498e
0032bfa835ce0f43a92ae0bbab6871cb
003a91841da04a5a31f808fb5c21538a
003d4dd6bd61221ebc0bfb9350db333f



In [27]:
error_wsis

[]

In [28]:
if len(error_wsis) > 0:
    logging.error("Slides processed with errors %s", str(error_wsis))

In [ ]:
len(patches_rows)

In [29]:
patches_df = pd.DataFrame(patches_rows, 
                          columns=['image_id', 'data_provider', 
                                   'isup_grade', 'gleason_score', 
                                   'y', 'x'] + [f"label{i}" for i in range(6)])

In [30]:
patches_df.to_csv(patches_csv_path, index=False)

In [215]:
import linecache
import sys

def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))


try:
    print(1/0)
except:
    PrintException()

EXCEPTION IN (<ipython-input-215-7f2a340f6b82>, LINE 15 "print(1/0)"): division by zero


In [70]:
df_row

image_id         9bc70f1bfc8dbca7c881616b0198edcb
data_provider                             radboud
isup_grade                                      4
gleason_score                                 4+4
Name: 6363, dtype: object

In [113]:
fname

'9bc70f1bfc8dbca7c881616b0198edcb.tiff'

In [126]:
os.path.join(wsi_path, fname)

'/mnt/HDDData/pdata/train_images/9bc70f1bfc8dbca7c881616b0198edcb.tiff'

In [137]:
result = subprocess.run([TIFFTILES2JPG_BIN, os.path.join(wsi_path, fname), '0', '../tmp/patches2/'], 
                         stdout=subprocess.PIPE)

result.returncode == 0

print(result.stdout.decode("utf-8"))

Image width: 9216, Image lenght: 32000
Tile width: 512, Tile lenght: 512
Tiles in width: 18, Tiles in lenght: 63
BitsPerSample: 8
SamplesPerPixel: 3
Compression: 7
Photometric: 2
JPEGTables size 289
Number of tiles: 1134
finished..............



0

Image width: 9216, Image lenght: 32000
Tile width: 512, Tile lenght: 512
Tiles in width: 18, Tiles in lenght: 63
BitsPerSample: 8
SamplesPerPixel: 3
Compression: 7
Photometric: 2
JPEGTables size 289
Number of tiles: 1134
finished..............

